I will now try to perform batch processing on input csv files, converting them to hdf5 using out-of-core processing.

In [1]:
%pylab
import DataSTORM.processors as proc
import DataSTORM.batch      as bat
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


Establish a filter and cleanup for testing.

In [2]:
filter1  = proc.Filter('loglikelihood', '<', 250)
cleanup  = proc.CleanUp()
pipeline = [cleanup, filter1]

In [3]:
inputDir = Path('../test-data/Centrioles/')
bp = bat.H5BatchProcessor(inputDir,
                          pipeline,
                          suffix = '_locResults_small.dat',
                          chunksize = None,
                          inputFileType = 'csv',
                          useSameFolder = True)

In [4]:
%time bp.go()

CPU times: user 604 ms, sys: 28 ms, total: 632 ms
Wall time: 632 ms


In [5]:
%time bp.goMerge()

Frame 889: 24 trajectories present
CPU times: user 3min 2s, sys: 5.35 s, total: 3min 7s
Wall time: 3min 4s


So it looks like linking two datasets with 80,000 and 100,000 frames (and that are relatively sparse) will take approximately 12 hours when done in serial.

In [6]:
test = Path('../test-data/Centrioles/FOV_1_noPB_1500mW_10ms_1/FOV_1_noPB_1500mW_10ms_1_MMStack_locResults_small_processed.h5')
test2 = pd.read_hdf(str(test), key = 'merged')

In [7]:
test2.describe()

,x,y,z,frame,loglikelihood,photons,bg,length
count,9877.000000,9877.000000,9877,9877.000000,9877.000000,9877.000000,9877.000000,9877.000000
mean,28557.021086,29254.265628,0,491.813202,116.001180,15391.987908,957.676042,4.404880
std,17561.910395,18578.362224,0,235.160640,43.086534,30445.832511,2476.715013,11.482319
min,133.361549,8.103771,0,100.000000,18.378000,1081.900000,152.120000,1.000000
25%,11221.807293,10732.208576,0,287.000000,85.059143,4067.700000,239.010000,1.000000
50%,26910.263572,27142.539840,0,485.000000,104.529067,8334.000000,500.520000,2.000000
75%,40961.458209,43556.076196,0,680.000000,137.114857,18087.700000,1144.560000,5.000000
max,64881.000000,64961.000000,0,939.000000,249.880000,1816196.500000,180336.790000,839.000000


In [ ]:
import trackpy as tp

In [ ]:
filename = Path('../test-data/Centrioles/FOV_1_noPB_1500mW_10ms_1/FOV_1_noPB_1500mW_10ms_1_MMStack_locResults_small_processed.h5')
store = tp.PandasHDFStoreSingleNode(str(filename), key = 'linked')
#store = pd.read_hdf(str(filename), key = 'linked', iterator = True, chunksize = None)

In [ ]:
store.store.remove('linked')

In [ ]:
store.store.groups

In [ ]:
test.agg({'loglikelihood' : 'mean', 'frame' : 'min'})

In [ ]:
temp = store.store.select_column('linked', 'particle').max()

In [ ]:
type(int(temp))